<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P2_summarization(seq2seq%2CT5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/MyDrive/nlp")
!pip install datasets
!pip install -U transformers
!pip install -U accelerate
!pip install rouge-chinese  #评估用
!pip install sentencepiece #T5模型需要
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM,DataCollatorForSeq2Seq,Seq2SeqTrainer,Seq2SeqTrainingArguments,pipeline
import numpy as np
from rouge_chinese import Rouge

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/mengzi-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("Langboat/mengzi-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/725k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
datasets=load_dataset("BelleGroup/train_1M_CN")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
datasets['train'][:10]

{'instruction': ['判断给定的文章是否符合语法规则。如果不符合，请提供修改建议。\n下面是一篇文章的开头: "为了探讨这个主题，本文将提供一系列数据和实例，以证明这一观点。"\n',
  '提供一个包含两个城市名称的列表，然后为这两个城市之间的距离提供一个近似值。\n1. 北京\n2. 上海\n',
  '解释人类尺度在绘画中的作用。\n',
  '给定一篇文章，纠正里面的语法错误。\n我去年很喜欢在公园里跑步，但因为最近天气太冷所以我不去了。\n',
  '根据以下输入生成相应的段落：\n[组织名]于今天宣布将收购[公司名]。该交易总价为[$总价]。[公司名]将继续保持独立，并在以前的管理团队领导下运营。',
  '列出现任意日历月份的日历。\n月份：五月\n',
  '求你告诉我一首悦耳的歌曲名字和演唱者。\n',
  '生成一个让我们发笑的笑话。\n',
  '将给定的一段文字转换为电子邮件格式，并添加收件人、主题和正文。\n文本: 亲爱的先生/女士，我希望与您分享我最近开发的一个产品。它是一款智能垃圾桶，能够自动分类各种废弃物，并将其回收。如果您对此感兴趣，请回复此电子邮件，我可以向您展示更多。\n收件人: john@example.com\n主题: 智能垃圾桶产品介绍\n',
  '给出一条英文新闻标题并要求将其翻译成中文。\n"SpaceX successfully launches 60 more Starlink satellites, bringing total to nearly 1k"\n'],
 'input': ['', '', '', '', '', '', '', '', '', ''],
 'output': ['这个开头符合语法规则。',
  '北京和上海之间的距离约为 1,295 公里。',
  '人类尺度在绘画中的作用是用来呈现逼真的透视和比例。通过考虑物体与人类身体的关系，艺术家可以创建出真实感和深度感，使观众更好地理解画作中的场景和物品。人类尺度可以让画作更加可信，并且帮助观众更容易地投入到画作的世界中。',
  '我去年很喜欢在公园里跑步，但由于最近天气太冷，所以我不去了。',
  '今天，[组织名]宣布他们将收购[公司名]，该交易总价为[$总价]。尽管发生了这样的变化，[公司名]将继续保持独立，继续保留

In [ ]:
def data_process(examples):
  contents = [i+j for i,j in zip(examples["instruction"],examples["input"])]
  inputs=tokenizer(contents, max_length=128, truncation=True)
  labels=tokenizer(text_target=examples["output"], max_length=64, truncation=True)
  inputs["labels"] = labels["input_ids"]
  return inputs

In [ ]:
datasets = datasets["train"].train_test_split(test_size=0.2, shuffle=True)

In [ ]:
ds = datasets.map(data_process,batched=True)
ds

Map:   0%|          | 0/733939 [00:00<?, ? examples/s]

Map:   0%|          | 0/183485 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 733939
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 183485
    })
})

In [ ]:
tokenizer.decode(ds["train"][0]["input_ids"])

'给定一个电影名和一个演员名,使用一种语句结构表达这个演员在这部电影中的角色和表现。 电影名: 《泰坦尼克号》 演员名: 莱昂纳多·迪卡普里奥</s>'

In [ ]:
tokenizer.decode(ds["train"][0]["labels"])

'莱昂纳多·迪卡普里奥在电影《泰坦尼克号》中扮演年轻的艺术家杰克·道森,从他独特的视角和勇敢的行为,真实而深刻地表达了电影主题的爱和生命的价值。</s>'

In [ ]:
rouge=Rouge()
#单文档摘要任务中，ROUGE-2, ROUGE-L, ROUGE-W,以及ROUGE-S表现很好 ；
# 在短摘要/标题式摘要任务中，ROUGE-1, ROUGE-L, ROUGE-W, ROUGE-SU4,以及ROUGE-SU9表现很棒 ；
# 在多文档摘要任务中，当停用词被排除在匹配之外时，ROUGE-1, ROUGE-2, ROUGE-S4,ROUGE-S9, ROUGE-SU4, 以及ROUGE-SU9表现不错 ；
# 排除停用词通常能改善相关性评估 使用多个reference(ideal 摘要)可以改善相关性评估。


def metric(preds):
  predictions,labels=preds
  decode_preds=tokenizer.batch_decode(predictions,skip_special_tokens=True)
  labels=np.where(labels != -100, labels, tokenizer.pad_token_id)
  decode_labels=tokenizer.batch_decode(labels,skip_special_tokens=True)
  decode_preds = ["".join(p) for p in decode_preds] #示例是all_hypothesis = ["哈 尔 滨 是 黑 龙 江 的 省 会"]
# all_references = [["哈 工 大 在 哈 尔 滨", "黑 龙 江 太 冷 了"]]字和字之间有空格，试一下没有空格能不能评分,结果是没区别

  decode_labels = ["".join(l) for l in decode_labels]
  scores = rouge.get_scores(decode_preds, decode_labels, avg=True)
  return {
        "rouge-1": scores["rouge-1"]["f"],
        "rouge-2": scores["rouge-2"]["f"],
        "rouge-l": scores["rouge-l"]["f"],
    }

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir="./summary",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    logging_steps=8,
    save_steps=10,
    eval_steps=10,
    metric_for_best_model="rouge-l",
    predict_with_generate=True #必须有这个，不然不能eval了
)#num_train_epochs=

In [ ]:
# args = Seq2SeqTrainingArguments(
#     output_dir="./summary",
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=8,
#     gradient_accumulation_steps=8,
#     logging_steps=8,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     metric_for_best_model="rouge-l",
#     predict_with_generate=True
# )

In [ ]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=metric,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer)
)

In [ ]:
trainer.train()

Step,Training Loss
8,3.806000
16,3.417400
24,3.399400


Checkpoint destination directory ./summary/checkpoint-20 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./summary/checkpoint-30 already exists and is non-empty.Saving will proceed but saved results may be invalid.


KeyboardInterrupt: 

In [ ]:
from transformers import pipeline
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/nlp/summary/checkpoint-30")
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)

In [ ]:
pipe(ds["test"][-1]["instruction"],  max_length=64,do_sample=True)#

[{'generated_text': '健康饮食 - 健康饮食 饮食之方式 -营养与健康饮食 《健康饮食 - 健康饮食的要点》 - 健康饮食 营养:饮食,建议和营养早餐:饮食。 饮食: 健康饮食是关键; 营养饮食是最重要的健康饮食非常重要,如早餐,'}]

In [ ]:
ds["test"][-1]["output"]

'营养与饮食习惯：打造健康生活的关键'

In [ ]:
a={"b":["wo","shi","ni","die"],"c":"a"}
t = ["content:"+i for i in a["b"]]

In [ ]:
t